# Lab 2
### Library fft() example

In [1]:
%use kmath
%use kandy

In [4]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.flow.first
import kotlinx.coroutines.flow.flowOf
import kotlinx.coroutines.flow.toList
import kotlinx.coroutines.runBlocking
import org.apache.commons.math3.transform.TransformType
import space.kscience.kmath.streaming.asFlow
import space.kscience.kmath.commons.transform.fft

val frequency = 100.0
val w = 2 * PI * frequency
fun signal(x: Double) = cos(w * x)

val partsCount = 0x001000
val range = 0.0 to 0.1
val step = (range.second - range.first) / partsCount
val args = (0..<partsCount).map { range.first + step * it }
val signalValues = args.map { signal(it) }

In [5]:
val signalFlow = flowOf(ListBuffer(signalValues))
val complexAmplitudes = runBlocking(context = Dispatchers.Default) {
    signalFlow.fft(direction = TransformType.FORWARD).first()
}

complexAmplitudes.toList()

[(3.441076229252936E-13 + i * 0.0), (2.5029194533599437E-13 + i * 1.4332660496371182E-13), (2.2639308632749232E-14 + i * 2.7315261473606275E-14), (3.068462315387342E-13 + i * -1.313613148084591E-13), (4.4352542404087233E-13 + i * -4.7525576233095866E-14), (2.932714269247102E-13 + i * 1.629013959523959E-13), (4.260946752884572E-13 + i * -8.325177775448829E-14), (4.694259537352847E-13 + i * -2.7910556201509745E-13), (1.0297358717002607E-12 + i * 2.0553764778210064E-14), (1.3390677285574301E-12 + i * 2.1806563858070824E-13), (2048.0 + i * 4.293454480830405E-12), (-1.2506919024702566E-12 + i * 1.444661700311203E-13), (-8.438768413995865E-13 + i * 5.3662628000874237E-14), (-3.2816328402876843E-13 + i * -2.449594589934754E-13), (-3.4808236548653264E-13 + i * 1.9259263255912153E-14), (-1.3063196678346467E-13 + i * 1.33964896018823E-13), (-2.2147392894388085E-13 + i * -7.91263120942672E-14), (-1.835728147009579E-13 + i * -4.2827644724905E-14), (1.561858973514931E-14 + i * 1.3171249629135938E-1

### dftSlow() example

In [6]:
import fourier.dftSlow

val complexAmplitudesDftSlow = dftSlow(signalValues)

complexAmplitudesDftSlow

[(2.6190161150907443E-13 + i * 8.64039742531908E-14), (1.7252865802674933E-13 + i * 8.64039742531908E-14), (2.6478819137309983E-13 + i * 8.64039742531908E-14), (2.87214696470528E-13 + i * 8.64039742531908E-14), (5.117017920497346E-13 + i * 8.64039742531908E-14), (4.757305660518796E-13 + i * 8.64039742531908E-14), (2.0472512574087887E-13 + i * 8.64039742531908E-14), (7.079892228034623E-13 + i * 8.64039742531908E-14), (1.6734391650174985E-12 + i * 8.64039742531908E-14), (1.4760415112391456E-12 + i * 8.64039742531908E-14), (2047.9999999999998 + i * 8.64039742531908E-14), (-4.007905118896815E-12 + i * 8.64039742531908E-14), (-1.8361978604275464E-12 + i * 8.64039742531908E-14), (8.104628079763643E-14 + i * 8.64039742531908E-14), (-8.923972671937008E-13 + i * 8.64039742531908E-14), (-1.1822764989233292E-12 + i * 8.64039742531908E-14), (-6.138423103152491E-13 + i * 8.64039742531908E-14), (6.150635556423367E-14 + i * 8.64039742531908E-14), (-3.0264679651281767E-13 + i * 8.64039742531908E-14), 

### Apply inverse transform to dftSlow result

In [13]:
import kotlinx.coroutines.flow.flowOf

val flow = flowOf(ListBuffer(complexAmplitudesDftSlow))

val signalRestored = runBlocking(context = Dispatchers.Default) {
    flow.fft(direction = TransformType.INVERSE).first().toList().map { it.re }
}

signalRestored

[1.0000000000009261, 0.9998823474547877, 0.9995294175016607, 0.9989412931870798, 0.9981181129003417, 0.9970600703395838, 0.9957674144679841, 0.9942404494534779, 0.9924795345989934, 0.9904850842568385, 0.9882575677311051, 0.9857975091676033, 0.983105487431407, 0.9801821359685366, 0.9770281426582776, 0.9736442496510869, 0.9700312531948199, 0.9661900034458992, 0.9621214042692467, 0.9578264130276052, 0.9533060403542092, 0.9485613499157675, 0.9435934581617645, 0.9384035340628862, 0.9329927988349208, 0.9273625256506743, 0.9215140393421264, 0.9154487160883531, 0.9091679830904897, 0.9026733182373391, 0.8959662497562304, 0.8890483558546354, 0.8819212643483448, 0.874586652278176, 0.8670462455156732, 0.8593018183570158, 0.8513551931053096, 0.8432082396418833, 0.8348628749863239, 0.8263210628456539, 0.8175848131516498, 0.8086561815882976, 0.7995372691081002, 0.7902302214376317, 0.7807372285720142, 0.7710605242617061, 0.7612023854840719, 0.7511651319096122, 0.7409511253547113, 0.7305627692275617, 0

### Compare with original signal

In [14]:
plot {
    line {
        x(args)
        y(signalValues)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="x7l0HP"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.0,2.44140625E-5,4.8828125E-5,7.324218750000001E-5,9.765625E-5,1.220703125E-4,1.4648437500000002E-4,1.7089843750000002E-4,1.953125E-4,2.197265625E-4,2.44140625E-4,2.685546875E-4,2.9296875000000004E-4,3.173828125E-4,3.4179687500000003E-4,3.662109375E-4,3.90625E-4,4.1503906250000004E-4,4.39453125E-4,4.6386718750000003E-4,4.8828125E-4,5.126953125E-4,5.37109375E-4,5.615234375000001E-4,5.859375000000001E-4,6.103515625E-4,6.34765625E-4,6.591796875E-4,6.835937500000001E-4,7.080078125000001E-4,7.32421875E-4,7.568359375E-4,7.8125E-4,8.056640625000001E-4,8.300781250000001E-4,8.544921875E-4,8.7890625E-4,9.033203125E-4,9.277343750000001E-4,9.521484375000001E-4,9.765625E-4,0.0010009765625000001,0.001025390625,0.0010498046875,0.00107421875,0.0010986328125,0.0011230468750000001,0.0011474609375,0.0011718750000000002,0.0011962890625,0.001220703125,0.0012451171875000001,0.00126953125,0.0012939453125000002,0.001318359375,0.0013427734375,0.0013671875000000001,0.0013916015625,0.0014160156250000002,0.0014404296875,0.00146484375,0.0014892578125000001,0.001513671875,0.0015380859375000002,0.0015625,0.0015869140625,0.0016113281250000001,0.0016357421875,0.0016601562500000002,0.0016845703125,0.001708984375,0.0017333984375000001,0.0017578125,0.0017822265625000002,0.001806640625,0.0018310546875,0.0018554687500000001,0.0018798828125,0.0019042968750000002,0.0019287109375,0.001953125,0.0019775390625,0.0020019531250000003,0.0020263671875,0.00205078125,0.0020751953125,0.002099609375,0.0021240234375000003,0.0021484375,0.0021728515625,0.002197265625,0.0022216796875,0.0022460937500000003,0.0022705078125,0.002294921875,0.0023193359375,0.0023437500000000003,0.0023681640625000003,0.002392578125,0.0024169921875,0.00244140625,0.0024658203125000003,0.0024902343750000003,0.0025146484375,0.0025390625,0.0025634765625,0.0025878906250000003,0.0026123046875000003,0.00263671875,0.0026611328125,0.002685546875,0.0027099609375000003,0.0027343750000000003,0.0027587890625,0.002783203125,0.0028076171875,0.0028320312500000003,0.0028564453125000003,0.002880859375,0.0029052734375,0.0029296875,0.0029541015625000003,0.0029785156250000003,0.0030029296875,0.00302734375,0.0030517578125,0.0030761718750000003,0.0031005859375000003,0.003125,0.0031494140625,0.003173828125,0.0031982421875000003,0.0032226562500000003,0.0032470703125,0.003271484375,0.0032958984375,0.0033203125000000003,0.0033447265625000003,0.003369140625,0.0033935546875,0.00341796875,0.0034423828125000003,0.0034667968750000003,0.0034912109375,0.003515625,0.0035400390625,0.0035644531250000003,0.0035888671875000003,0.00361328125,0.0036376953125,0.003662109375,0.0036865234375000003,0.0037109375000000003,0.0037353515625,0.003759765625,0.0037841796875,0.0038085937500000003,0.0038330078125000003,0.003857421875,0.0038818359375,0.00390625,0.0039306640625,0.003955078125,0.0039794921875,0.0040039062500000005,0.0040283203125,0.004052734375,0.0040771484375,0.0041015625,0.0041259765625000005,0.004150390625,0.0041748046875,0.00419921875,0.0042236328125,0.0042480468750000005,0.0042724609375,0.004296875,0.0043212890625,0.004345703125,0.0043701171875000005,0.00439453125,0.0044189453125,0.004443359375,0.0044677734375,0.0044921875000000005,0.0045166015625,0.004541015625,0.0045654296875,0.00458984375,0.0046142578125000005,0.004638671875,0.0046630859375,0.004687500000000001,0.0047119140625,0.0047363281250000005,0.0047607421875,0.00478515625,0.004809570312500001,0.

In [15]:
plot {
    line {
        x(args)
        y(signalRestored)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="2sWcTQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.0,2.44140625E-5,4.8828125E-5,7.324218750000001E-5,9.765625E-5,1.220703125E-4,1.4648437500000002E-4,1.7089843750000002E-4,1.953125E-4,2.197265625E-4,2.44140625E-4,2.685546875E-4,2.9296875000000004E-4,3.173828125E-4,3.4179687500000003E-4,3.662109375E-4,3.90625E-4,4.1503906250000004E-4,4.39453125E-4,4.6386718750000003E-4,4.8828125E-4,5.126953125E-4,5.37109375E-4,5.615234375000001E-4,5.859375000000001E-4,6.103515625E-4,6.34765625E-4,6.591796875E-4,6.835937500000001E-4,7.080078125000001E-4,7.32421875E-4,7.568359375E-4,7.8125E-4,8.056640625000001E-4,8.300781250000001E-4,8.544921875E-4,8.7890625E-4,9.033203125E-4,9.277343750000001E-4,9.521484375000001E-4,9.765625E-4,0.0010009765625000001,0.001025390625,0.0010498046875,0.00107421875,0.0010986328125,0.0011230468750000001,0.0011474609375,0.0011718750000000002,0.0011962890625,0.001220703125,0.0012451171875000001,0.00126953125,0.0012939453125000002,0.001318359375,0.0013427734375,0.0013671875000000001,0.0013916015625,0.0014160156250000002,0.0014404296875,0.00146484375,0.0014892578125000001,0.001513671875,0.0015380859375000002,0.0015625,0.0015869140625,0.0016113281250000001,0.0016357421875,0.0016601562500000002,0.0016845703125,0.001708984375,0.0017333984375000001,0.0017578125,0.0017822265625000002,0.001806640625,0.0018310546875,0.0018554687500000001,0.0018798828125,0.0019042968750000002,0.0019287109375,0.001953125,0.0019775390625,0.0020019531250000003,0.0020263671875,0.00205078125,0.0020751953125,0.002099609375,0.0021240234375000003,0.0021484375,0.0021728515625,0.002197265625,0.0022216796875,0.0022460937500000003,0.0022705078125,0.002294921875,0.0023193359375,0.0023437500000000003,0.0023681640625000003,0.002392578125,0.0024169921875,0.00244140625,0.0024658203125000003,0.0024902343750000003,0.0025146484375,0.0025390625,0.0025634765625,0.0025878906250000003,0.0026123046875000003,0.00263671875,0.0026611328125,0.002685546875,0.0027099609375000003,0.0027343750000000003,0.0027587890625,0.002783203125,0.0028076171875,0.0028320312500000003,0.0028564453125000003,0.002880859375,0.0029052734375,0.0029296875,0.0029541015625000003,0.0029785156250000003,0.0030029296875,0.00302734375,0.0030517578125,0.0030761718750000003,0.0031005859375000003,0.003125,0.0031494140625,0.003173828125,0.0031982421875000003,0.0032226562500000003,0.0032470703125,0.003271484375,0.0032958984375,0.0033203125000000003,0.0033447265625000003,0.003369140625,0.0033935546875,0.00341796875,0.0034423828125000003,0.0034667968750000003,0.0034912109375,0.003515625,0.0035400390625,0.0035644531250000003,0.0035888671875000003,0.00361328125,0.0036376953125,0.003662109375,0.0036865234375000003,0.0037109375000000003,0.0037353515625,0.003759765625,0.0037841796875,0.0038085937500000003,0.0038330078125000003,0.003857421875,0.0038818359375,0.00390625,0.0039306640625,0.003955078125,0.0039794921875,0.0040039062500000005,0.0040283203125,0.004052734375,0.0040771484375,0.0041015625,0.0041259765625000005,0.004150390625,0.0041748046875,0.00419921875,0.0042236328125,0.0042480468750000005,0.0042724609375,0.004296875,0.0043212890625,0.004345703125,0.0043701171875000005,0.00439453125,0.0044189453125,0.004443359375,0.0044677734375,0.0044921875000000005,0.0045166015625,0.004541015625,0.0045654296875,0.00458984375,0.0046142578125000005,0.004638671875,0.0046630859375,0.004687500000000001,0.0047119140625,0.0047363281250000005,0.0047607421875,0.00478515625,0.004809570312500001,0.